In [1]:
import pandas as pd
import numpy as np
import re
import gc
import os
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD
import warnings
warnings.filterwarnings("ignore")

In [1]:
DATA_PATH = '../data/data_set_0926/'
FEAT_PATH = './features/'

In [2]:
tic = time.time()
invite_info = pd.read_table(os.path.join(DATA_PATH, 'invite_info_0926.txt'), header=None)
invite_info.columns = ['问题ID','用户ID','邀请创建时间','邀请是否被回答']
    
invite_info_evaluate_A = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_1_0926.txt'), header=None)
invite_info_evaluate_A.columns =  ['问题ID','用户ID','邀请创建时间']

invite_info_evaluate_B = pd.read_table(os.path.join(DATA_PATH, 'invite_info_evaluate_2_0926.txt'), header=None)
invite_info_evaluate_B.columns =  ['问题ID','用户ID','邀请创建时间']

question_info = pd.read_table(os.path.join(DATA_PATH, 'question_info_0926.txt'), header=None)
question_info.columns = ['问题ID',  '问题创建时间' , '问题标题的单字编码序列' , '问题标题的切词编码序列' , '问题描述的单字编码序列',  '问题描述的词编码序列' , '问题绑定的话题ID']

member_info = pd.read_table(os.path.join(DATA_PATH, 'member_info_0926.txt'), header=None)
member_info.columns = ['用户ID','性别','创作关键词的编码序列','创作数量等级','创作热度等级','注册类型','注册平台','访问频率','用户二分类特征A','用户二分类特征B','用户二分类特征C','用户二分类特征D','用户二分类特征E','用户分类特征A','用户分类特征B','用户分类特征C','用户分类特征D','用户分类特征E','用户的盐值分数','用户关注的话题','用户感兴趣的话题']

oversample = False

if oversample:
    data = pd.concat([invite_info, invite_info_evaluate_B, invite_info_evaluate_A], axis=0)
else:
    data = pd.concat([invite_info, invite_info_evaluate_B], axis=0)

data = data.merge(member_info, on='用户ID', how='left').merge(question_info, on='问题ID', how='left')

print("Used time: %d s" % (time.time()-tic))

Used time: 101 s


In [3]:
tic = time.time()
data['用户关注的话题_len'] = data['用户关注的话题'].apply(lambda x:len(x.split(',')))
data['用户感兴趣的话题_len'] = data['用户感兴趣的话题'].apply(lambda x:len(x.split(',')))
data['问题绑定的话题ID_len'] = data['问题绑定的话题ID'].apply(lambda x:len(x.split(',')))

tmp = data[['用户关注的话题_len','用户感兴趣的话题_len','问题绑定的话题ID_len']]
print("Used time: %d s" % (time.time()-tic))

Used time: 52 s


In [4]:
tmp.to_pickle(os.path.join(FEAT_PATH, 'len_feat.pickle'))
print("Feature Saved, shape:",tmp.shape)

Feature Saved, shape: (10630880, 3)
